In [1]:
# !wget https://raw.githubusercontent.com/mesolitica/malaysian-dataset/master/llm-benchmark/BM-pt3/BM-A-pt3

In [2]:
# import os

# os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from transformers.trainer_utils import get_last_checkpoint

#latest = get_last_checkpoint("fpf-7b-instructions-16k-call")

/home/cybertron/anaconda3/envs/sealion/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# tokenizer = AutoTokenizer.from_pretrained(latest)
# model = AutoModelForCausalLM.from_pretrained(
#     latest, 
#     use_flash_attention_2 = True, 
#     torch_dtype = torch.float16,
#     device_map="cuda:0"
# )
tokenizer = AutoTokenizer.from_pretrained('mesolitica/malaysian-llama2-7b-32k-instructions-v2', trust_remote_code=True)
kwargs = {'device_map': 'auto', 'torch_dtype': torch.float16}
model= AutoModelForCausalLM.from_pretrained('mesolitica/malaysian-llama2-7b-32k-instructions-v2', **kwargs)

Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████| 3/3 [00:04<00:00,  1.55s/it]


In [5]:
from tqdm import tqdm
import os
import random
torch.manual_seed(404)
random.seed(404)

In [6]:

data_folder = '/media/cybertron/fa54fcb6-b5e1-492e-978a-6389519c168a/llm-benchmarks/sejarah-spm/data/'
with open(data_folder+ 'sejarah_spm') as fopen:
    text = fopen.read()
    
    
questions = []
for t in text.split('no: ')[1:]:
    t = t.strip()
    no = t.split('\n')[0]
    soalan = t.split('soalan:')[1].split('jawapan:')[0].strip()
    jawapan = t.split('jawapan: ')[1].split(',')[0].strip()
    data = {
        'no': no,
        'soalan': soalan,
        'jawapan': jawapan,
    }
    print(data)
    questions.append(data)

{'no': '1', 'soalan': 'Apakah yang dimaksudkan dengan wilayah pengaruh?\nA. Kawasan tanah jajahan dan takluk raja\nB. Kawasan yang dinaungi oleh raja dan kerabatnya\nC. Kawasan yang rakyatnya menerima dan memperakui pemerintahan seseorang raja\nD. Kawasan yang rakyatnya menganggap raja sebagai tuhan atau wakil tuhan', 'jawapan': 'C'}
{'no': '2', 'soalan': 'Bagaimanakah Raja dalam kerajaan Alam Melayu memperkukuhkan kedudukannya?\nA. Ketaatan rakyat kepada raja\nB. Rakyat dianggap suci\nC. Rakyat menyembah raja\nD. Raja menjalankan kerahan tenaga', 'jawapan': 'A'}
{'no': '3', 'soalan': 'Apakah faktor perbezaan idea nasionalisme antara negara dengan zaman?\nA. Politik\nB. Darjat\nC. Keturunan\nD. Ekonomi', 'jawapan': 'C'}
{'no': '4', 'soalan': "Maklumat berikut berkaitan dengan pendapat tokoh mengenai nasionalisme. \nKita hendak menzahirkan suatu bangsa yang lebih tegap dan kuat, boleh\nberdiri di atas tapak kaki kita sendiri dan mengambil tempat kita yang\nsebenar di sisi bangsa-bangsa 

In [7]:
arange = set(range(len(questions)))

In [8]:
def convert_prompt(row, answer = False):
    if answer:
        prompt = f"""
soalan: {row['soalan']}
jawapan: {row['jawapan']}
    """
    else:
        prompt = f"""
soalan: {row['soalan']}
jawapan:
    """
    return prompt.strip()

In [9]:
def parse_llama_chat(messages):

    system = messages[0]['content']
    user_query = messages[-1]['content']

    users, assistants = [], []
    for q in messages[1:-1]:
        if q['role'] == 'user':
            users.append(q['content'])
        elif q['role'] == 'assistant':
            assistants.append(q['content'])

    texts = [f'<s>[INST] <<SYS>>\n{system}\n<</SYS>>\n\n']
    for u, a in zip(users, assistants):
        texts.append(f'{u.strip()} [/INST] {a.strip()} </s><s>[INST] ')
    texts.append(f'{user_query.strip()} [/INST]')
    prompt = ''.join(texts).strip()
    return prompt

In [10]:
i = 1
prompts = []

prompts.append(convert_prompt(questions[i]))
prompt = '\n\n'.join(prompts).strip()
print(prompt)

soalan: Bagaimanakah Raja dalam kerajaan Alam Melayu memperkukuhkan kedudukannya?
A. Ketaatan rakyat kepada raja
B. Rakyat dianggap suci
C. Rakyat menyembah raja
D. Raja menjalankan kerahan tenaga
jawapan:


In [11]:
messages = [
    {'role': 'system', 'content': 'awak adalah AI yang mampu jawab segala soalan'},
    {'role': 'user', 'content': prompt}
]
prompt = parse_llama_chat(messages)
print("prompt :",prompt)
inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
inputs

prompt : <s>[INST] <<SYS>>
awak adalah AI yang mampu jawab segala soalan
<</SYS>>

soalan: Bagaimanakah Raja dalam kerajaan Alam Melayu memperkukuhkan kedudukannya?
A. Ketaatan rakyat kepada raja
B. Rakyat dianggap suci
C. Rakyat menyembah raja
D. Raja menjalankan kerahan tenaga
jawapan: [/INST]


{'input_ids': tensor([[    1,   518, 25580, 29962,  3532, 14816, 29903,  6778,    13,  1450,
           557,   594,   284,   801,   319, 29902,   343,   574,   286,  1160,
         29884,   432,  1450,   370,  2377,  2883,   577,   284,   273,    13,
         29966,   829, 14816, 29903,  6778,    13,    13,   578,   284,   273,
         29901,   350,  7781, 25895,   557,   801,   390,  9919,  2959,   314,
         13023,  9919,   273,   838,   314,  6286,   388, 29884,  2626,   546,
          2120,  2120, 29882, 11052,   413,   287,   566,  2679,   812,  3761,
         29973,    13, 29909, 29889,   476,  1187, 23402,  1153,  3459,   271,
           413,  1022,  1114,  1153,  1764,    13, 29933, 29889, 29350, 29891,
           271,   652,   574, 29887,   481,   480,   455,    13, 29907, 29889,
         29350, 29891,   271,  1757, 29891,  1590,   801,  1153,  1764,    13,
         29928, 29889,   390,  9919,  1757, 12429,   804,   273, 13023,   801,
           273,  3006,  7781,    13, 2

In [12]:
generate_kwargs = dict(
    inputs,
    max_new_tokens=100,
    top_p=0.95,
    top_k=50,
    temperature=0.1,
    do_sample=True,
    num_beams=1,
    repetition_penalty=1.05,
)
r = model.generate(**generate_kwargs)
print(tokenizer.decode(r[0]).split('jawapan:')[1].strip().split())

['[/INST]', 'A', '</s>']


In [13]:
tokenizer.decode(r[0])

'<s> [INST] <<SYS>>\nawak adalah AI yang mampu jawab segala soalan\n<</SYS>>\n\nsoalan: Bagaimanakah Raja dalam kerajaan Alam Melayu memperkukuhkan kedudukannya?\nA. Ketaatan rakyat kepada raja\nB. Rakyat dianggap suci\nC. Rakyat menyembah raja\nD. Raja menjalankan kerahan tenaga\njawapan: [/INST] A </s>'

In [18]:
for i in tqdm(range(len(questions))):
    
    prompts = []
    prompts.append(convert_prompt(questions[i]))
    prompt = '\n\n'.join(prompts)
    messages = [
        {'role': 'system', 'content': 'awak adalah AI yang mampu jawab segala soalan'},
        {'role': 'user', 'content': prompt}
    ]
    prompt = parse_llama_chat(messages)
    # print(prompt)
    # break
    inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')
    repeat = []
    for _ in range(5):
        try:
            generate_kwargs = dict(
                inputs,
                max_new_tokens=100,
                top_p=0.95,
                top_k=50,
                temperature=0.3,
                do_sample=True,
                num_beams=1,
                repetition_penalty=1.05,
            )
            r = model.generate(**generate_kwargs)
            splitted = tokenizer.decode(r[0]).split('[/INST]')[1].strip().replace('</s>', '').replace('.', '').replace(',', '').strip().split()
            splitted = [t for t in splitted if len(t) == 1]
            repeat.append(splitted[0])
    
        except Exception as e:
            pass
    
    questions[i]['output'] = repeat

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:39<00:00,  1.96s/it]


In [15]:
def most_common(l):
    return max(set(l), key=l.count)

In [16]:
filtered = [q for q in questions if 'output' in q and len(q['output'])]
correct = 0
for q in filtered:
    correct += most_common(q['output']) == q['jawapan']
(correct / len(filtered)) * 100

50.0